In [5]:
from ultralytics import YOLO
import os
import cv2
from pathlib import Path

In [6]:
model = YOLO("runs/detect/train3/weights/best.pt")

In [7]:
dataset_root = "MRI Scans"
output_root = "cropped_images"

for split in ["Testing", "Training"]:
    image_dir = Path(dataset_root) / split
    output_dir = Path(output_root) / split
    output_dir.mkdir(parents=True, exist_ok=True)

    for tumor_type in ["glioma", "meningioma","pituitary","no_tumor"]:
        input_folder = image_dir / tumor_type
        output_folder = output_dir / tumor_type
        output_folder.mkdir(parents=True, exist_ok=True)

        for img_path in input_folder.rglob("*.jpg"):
            results = model(str(img_path), device=0)  # run on GPU if available
            for i, box in enumerate(results[0].boxes.xyxy.cpu().numpy()):
                x1, y1, x2, y2 = map(int, box[:4])
                img = cv2.imread(str(img_path))
                crop = img[y1:y2, x1:x2]
                crop_name = f"{img_path.stem}_crop_{i}.jpg"
                cv2.imwrite(str(output_folder / crop_name), crop)



image 1/1 g:\My Programming files\Python dev\Machine Learning Project Final Year\MRI Scans\Testing\glioma\image(1).jpg: 640x512 1 tumor, 11.2ms
Speed: 1.2ms preprocess, 11.2ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 512)

image 1/1 g:\My Programming files\Python dev\Machine Learning Project Final Year\MRI Scans\Testing\glioma\image(10).jpg: 640x608 1 tumor, 5.6ms
Speed: 1.9ms preprocess, 5.6ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 608)

image 1/1 g:\My Programming files\Python dev\Machine Learning Project Final Year\MRI Scans\Testing\glioma\image(100).jpg: 640x576 1 tumor, 11.7ms
Speed: 1.7ms preprocess, 11.7ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 576)

image 1/1 g:\My Programming files\Python dev\Machine Learning Project Final Year\MRI Scans\Testing\glioma\image(11).jpg: 640x608 1 tumor, 5.6ms
Speed: 1.8ms preprocess, 5.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 608)

image 1/1 g:\My Programming fil

In [8]:
import shutil

cropped_dir = Path("cropped_images")
class_output_dir = Path("classification_dataset")

for split in ["training", "testing"]:
    for tumor_type in ["glioma", "meningioma", "pituitary", "no_tumor"]:
        input_dir = cropped_dir / split / tumor_type
        output_dir = class_output_dir / split / tumor_type
        output_dir.mkdir(parents=True, exist_ok=True)

        for img_path in input_dir.glob("*.jpg"):
            shutil.copy(str(img_path), str(output_dir / img_path.name))